In [87]:
# import math
# C1 = 1.0
# C2 = 1.0
# C3 = 1.0
# C4 = 1.0
# DeviceLag = 0.0
# FrameInterval = 1.0/90.0
# MachineLag = DeviceLag + FrameInterval*0.75
#
# # we calculate index of difficulty for 3 dimensions, W should be a tuple containing the same value thrice, thus we can simplify this expression
# def ID_3D(D, W):
#     return math.log2(D/min(W) + C4)
#
# # as the target is a round ball rather than some other shape, we know W1=W2=W3, we can simply not use a tuple for W
# def ID(D, W):
#     return math.log2(D/W+C4)
#
# # we can simply use the 1D fitts' law as widths are equal
# def fitts_3D(D, W):
#     return C1 + C2 * (C3 + MachineLag) * ID_3D(D, W)
#
# def fitts(D, W):
#     return C1 + C2 * (C3 + MachineLag) * ID(D, W)
#


In [88]:
import os, glob, math
from numpy import mean
from pathlib import Path

latencies = [0, 150, 300, 450, 600, 750]

def distance(v1, v2):
    return math.sqrt((v1[0] - v2[0])**2 + (v1[1] - v2[1])**2 + (v1[2] - v2[2])**2)

def ingest():
    experiment_data = {}
    path = '../experiment_data'
    dir_names = []
    for item in os.scandir(path):
        if item.is_dir():
            dir_names.append(item.name)
    for dir in dir_names:
        experiment_data[dir] = []
        for filename in glob.glob(os.path.join(path + '/' + dir, '*.txt')):
           with open(os.path.join(os.getcwd(), filename), 'r') as f:
               current_data = {'Experiment': eval(f.readline()[12:]), 'Latency': eval(f.readline()[9:]),
                               'Source': eval(f.readline()[8:]), 'SourceSize': eval(f.readline()[12:]),
                               'Target': eval(f.readline()[8:]), 'TargetSize': eval(f.readline()[12:])}
               f.readline()
               f.readline()
               f.readline()
               lines = f.readlines()
               data = {'time': [], 'position': []}
               prev_time = -1
               for line in lines:
                   line_split = line.split(';')
                   time = float(line_split[0])
                   position = eval(line_split[1])
                   if time == prev_time:
                       data['position'][-1] = tuple(mean((position, data['position'][-1]), axis=0))
                   else:
                       data['time'].append(time)
                       data['position'].append(position)
                   if len(data['position']) >= 2 and data['time'][-1] == data['time'][-2]:
                       if data['position'][-1] != data['position'][-2]:
                           print('mismatch!', filename, data['time'][-1], data['time'][-2], data['position'][-1], data['position'][-2])
                   prev_time = time
               current_data['data'] = data
               current_data['final_position'] = data['position'][-1]
               current_data['distance'] = distance(data['position'][-1], current_data['Target'])
               current_data['success'] = current_data['distance'] <= current_data['TargetSize'] / 2
               experiment_data[dir].append(current_data)
               f.close()
    return experiment_data

experiments = ingest()
experiments

{'niels': [{'Experiment': 0,
   'Latency': 0,
   'Source': (0.35, 0.84, 0.5),
   'SourceSize': 0.075,
   'Target': (0.65, 0.16, 0.5),
   'TargetSize': 0.075,
   'data': {'time': [46.5646,
     46.5757,
     46.58,
     46.587,
     46.598,
     46.6,
     46.6093,
     46.62,
     46.6202,
     46.6312,
     46.64,
     46.6424,
     46.6534,
     46.66,
     46.6644,
     46.6758,
     46.68,
     46.6866,
     46.6976,
     46.7,
     46.709,
     46.72,
     46.7311,
     46.74,
     46.7423,
     46.7536,
     46.76,
     46.7649,
     46.7759,
     46.78,
     46.7869,
     46.798,
     46.8,
     46.8091,
     46.82,
     46.8201,
     46.8311,
     46.84,
     46.8424,
     46.8534,
     46.86,
     46.8644,
     46.8758,
     46.88,
     46.8866,
     46.8978,
     46.9,
     46.9091,
     46.92,
     46.931,
     46.94,
     46.9425,
     46.9534,
     46.96,
     46.9645,
     46.9757,
     46.98,
     46.9867,
     46.9979,
     47.0,
     47.0089,
     47.0199,
     47.02,


In [89]:
def first_analysis(experiment_data):
    experiment_results = {}
    for person in experiment_data.items():
        times_latencies = {0:[], 150:[], 300:[], 450:[], 600:[], 750:[]}
        avg_time_latency = {}
        # times_experiments = [[]*9]
        times_experiments = []
        success_experiments = []
        distance_experiments = []
        off_experiments = []
        for n in range(9):
            times_experiments.append([])
            success_experiments.append([])
            distance_experiments.append([])
            off_experiments.append([])
        avg_time_experiment = []
        for data in person[1]:
            times_latencies[data['Latency']].append(data['data']['time'][-1] - data['data']['time'][0])
            times_experiments[data['Experiment']].append(data['data']['time'][-1] - data['data']['time'][0])
            success_experiments[data['Experiment']].append(data['success'])
            distance_experiments[data['Experiment']].append(data['distance'])
            off_experiments[data['Experiment']].append(data['distance']-data['TargetSize']/2)
        for latency in times_latencies.keys():
            avg_time_latency[latency] = sum(times_latencies[latency])/len(times_latencies[latency])
        for n in range(9):
            avg_time_experiment.append(sum(times_experiments[n])/len(times_experiments[n]))
        experiment_results[person[0]] = {'time_per_latency' : times_latencies, 'time_per_experiment': times_experiments, 'avg_time_per_latency': avg_time_latency, 'avg_time_per_experiment': avg_time_experiment, 'success': success_experiments, 'distance': distance_experiments, 'off': off_experiments}
    return experiment_results

results = first_analysis(experiments)
results

{'niels': {'time_per_latency': {0: [0.8353999999999999,
    1.5690000000000026,
    0.5889000000000024,
    0.7111000000000018,
    0.7663000000000011,
    0.6234000000000037,
    1.334299999999999,
    1.8110999999999962,
    1.0967999999999947],
   150: [0.8323000000000036,
    1.6030999999999977,
    0.9450000000000074,
    1.0178999999999974,
    1.2776999999999958,
    0.8436000000000092,
    1.4116999999999962,
    1.7356999999999942,
    1.077699999999993],
   300: [1.9446999999999974,
    1.9213000000000022,
    1.1155000000000115,
    1.439300000000003,
    1.8109000000000037,
    1.053200000000004,
    1.464100000000002,
    2.3663999999999987,
    1.2331999999999965],
   450: [1.6530000000000058,
    2.3671000000000078,
    1.5063000000000102,
    1.50030000000001,
    2.066599999999994,
    1.3001000000000147,
    2.542100000000005,
    3.3223000000000127,
    2.0600000000000023],
   600: [2.2335999999999956,
    3.038099999999986,
    2.4776999999999987,
    1.726100000000

In [90]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
names = results.keys()
timesdf = pd.DataFrame()

for name in names:
    df = pd.DataFrame(results[name]['time_per_latency'])
    df['name'] = name
    timeslatencydf = pd.concat([timesdf, df])

for name in names:
    df = pd.DataFrame(results[name]['time_per_experiment'])
    df['name'] = name
    timesexperimentdf = pd.concat([timesdf, df])

latencydf = pd.DataFrame(columns=latencies)

for latency in latencies:
    avg_times = []
    for experiment in range(9):
        times = []
        for name in names:
            times.append(results[name]['time_per_latency'][latency][experiment])
        avg_times.append(np.mean(times))
    latencydf[latency] = pd.Series(avg_times)
latencydf



,0,150,300,450,600,750
0,0.889450,1.145500,1.580875,2.409525,2.094275,3.480675
1,1.150200,1.619650,2.022025,2.914500,2.901250,3.156025
2,0.740675,1.140725,1.215450,1.736325,1.980175,2.834275
3,0.724225,1.123325,1.530975,1.631625,2.099325,2.345875
4,0.935850,1.365275,1.863825,1.919325,2.672475,3.057725
5,0.642000,0.897275,1.270000,1.668850,1.753875,1.939325
6,1.105375,1.592475,1.965775,2.925950,4.300300,5.087325
7,1.403100,1.778325,2.401200,3.168875,4.485525,4.837500
8,1.046525,1.241250,1.580825,3.055600,2.976425,3.844050


In [94]:
successdf = pd.DataFrame()
for person in results.keys():
    df = pd.DataFrame(results[person]['success'])
    df.columns = latencies
    df['name'] = person
    successdf = pd.concat([successdf, df])
successdf

,0,150,300,450,600,750,name
0,False,False,False,False,True,True,niels
1,True,True,False,False,True,False,niels
2,True,False,True,False,True,True,niels
3,True,False,False,False,False,True,niels
4,True,True,True,True,False,True,niels
5,False,False,False,True,False,False,niels
6,False,False,False,False,False,False,niels
7,False,True,False,True,False,True,niels
8,False,False,False,False,True,False,niels
0,True,False,True,True,False,False,ralf


In [92]:
distancesdf = pd.DataFrame()
for person in results.keys():
    df = pd.DataFrame(results[person]['distance'])
    df.columns = latencies
    df['name'] = person
    distancesdf = pd.concat([distancesdf, df])
distancesdf

,0,150,300,450,600,750,name
0,0.118743,0.100499,0.064031,0.067082,0.031623,0.022361,niels
1,0.017321,0.036056,0.054772,0.050990,0.036056,0.054772,niels
2,0.037417,0.064807,0.031623,0.076811,0.030000,0.031623,niels
3,0.041231,0.155242,0.116619,0.100499,0.081240,0.050990,niels
4,0.041231,0.061644,0.046904,0.041231,0.078740,0.053852,niels
5,0.081240,0.112250,0.091104,0.050000,0.084853,0.064031,niels
6,0.014142,0.028284,0.022361,0.031623,0.014142,0.017321,niels
7,0.014142,0.010000,0.017321,0.010000,0.028284,0.010000,niels
8,0.020000,0.031623,0.028284,0.030000,0.000000,0.017321,niels
0,0.014142,0.041231,0.031623,0.030000,0.050990,0.073485,ralf


In [93]:
offdf = pd.DataFrame()
for person in results.keys():
    df = pd.DataFrame(results[person]['off'])
    df.columns = latencies
    df['name'] = person
    offdf = pd.concat([offdf, df])
offdf

,0,150,300,450,600,750,name
0,0.081243,0.062999,0.026531,0.029582,-0.005877,-0.015139,niels
1,-0.020179,-0.001444,0.017272,0.013490,-0.001444,0.017272,niels
2,-0.000083,0.027307,-0.005877,0.039311,-0.007500,-0.005877,niels
3,-0.021269,0.092742,0.054119,0.037999,0.018740,-0.011510,niels
4,-0.021269,-0.000856,-0.015596,-0.021269,0.016240,-0.008648,niels
5,0.018740,0.049750,0.028604,-0.012500,0.022353,0.001531,niels
6,0.001642,0.015784,0.009861,0.019123,0.001642,0.004821,niels
7,0.001642,-0.002500,0.004821,-0.002500,0.015784,-0.002500,niels
8,0.007500,0.019123,0.015784,0.017500,-0.012500,0.004821,niels
0,-0.023358,0.003731,-0.005877,-0.007500,0.013490,0.035985,ralf
